## Debugging con GDB

- [GDB: The GNU Project Debugger](https://www.sourceware.org/gdb/)
- Crítico para resolver problemas complejos en programas de C.
- CLI (comando gdb / gdb -tui) o GUI (gdbgui, editors, etc).
- Manual [Debugging with GDB](https://sourceware.org/gdb/current/onlinedocs/gdb/)

![gnu_gdb_logo](img/gnu-gdb-logo.jpg)

### Builds para debug en CMake
- En programas construidos con CMake, debemos ajustar el tipo de build:
    - Mantener los símbolos de debug.
    - Desactivar optimizaciones para mantener el flujo original.
    - Comando: `cmake -DCMAKE_BUILD_TYPE=Debug ..`

### gdb CLI
- Correr gdb con un ejecutable (incluyendo argumentos)
   - `gdb --args <executable>`
   - Ej: `gdb --args ./tools/sensor_cmds/sensor_cmds -c ../config/sensors_cfg.json`
- Correr gdb TUI (Text User Interface)
    - Manual)[https://sourceware.org/gdb/onlinedocs/gdb/TUI.html]
    - 


### gdb GUI
- gdbgui: Frontend web para GDB
- Instalación
    - `python3 -m venv gdb_venv`
    - `source gdb_venv/bin/activate`
    - `pip install werkzeug==2.0.0 gdbgui`
-  Levantar servidor
    - `gdbgui --host 0.0.0.0`


### Comandos básicos
- Mostrar ayuda
    - `help`
- Iniciar el programa
    - `start`
- Print symbol
    - `p <symbol>`

### Breakpoints
- Setear breakpoint en ubicación
  - `br <symbol/file>`
     - Ej: `br zmq_server_rep_pthread.c:63`
     - Ej: `br zframe_recv`
- Obtener breakpoints actuales
  - `info br`
- Eliminar breakpoints
  - `delete br [<br_num>]`

### Watchpoints
- Obtener watchpoints actuales
  - `info watch`
- Setear watchpoints en símbolo o condición
  - `watch <symbol>`
    - Ej: `watch rep->val_a`

### Memoria
- Examinar memoria
    - `x/nfu addr`
        - Ej: `x/10xw 0x7ffff0006740`

### Debug de backtrace
- Obtener backtrace para el thread actual
   - `bt`
- Obtener información del frame actual
   - `frame`
- Setear frame actual
    - `frame <frame_num>`

### Debug de instrucciones
- Revisar código de ensamblador para una función o dirección
  - `disassemble <symbol>`
    - Ej: `disassemble msg_server_fn`
- Print CPU registers
  - `info reg`

### Debug de threads
- Obtener los threads actuales
  - `info threads`
- Aplicar comando a todos los threads
  - `thread apply all <command>`
    - Ej: `thread apply all bt`
  - Setear el thread actual
    - `thread <thread_num>`

### Práctica GDB
#### Ejercicio 1 (sensor commands)
- `gdb --args ./tools/sensor_cmds/sensor_cmds -c ../config/sensors_cfg.json`
- `start`
- `br ssr_read_cmd_exec_fn`
- `continue`
- step (s) hasta llegar a sensor read
- next (n) hasta llegar a `return state->value`

- `print *info`
- `print *state`
- `bt`
- `frame 2`
- `print *ssr`
- `frame 4`
- `print *cmd_runner`
- `print *cmd`       
- `frame 0`
- `disassemble level_sensor_read`

- `print reg`
- `thread apply all bt`
- Saltar al thread de main
  - `thread 1`
- `bt`
- `frame 1`
- `print *cmd_runner`
- `frame 2`
- `print commands`

- Revisar sensor dentro del comando 2
  - `p *(((struct ssr_read_cmd_data *)(commands[1]->data))->ssr)`
- Saltar al thread de command runner otra vez
  - `thread 2`
- `bt`
- `delete br`
- `continue`

### Ejercicio 2
- Comentar check (`cmd->execute == NULL`) en `command_runner.c:174`
- Reconstruir sensor_commands
- `gdb --args ./tools/sensor_cmds/sensor_cmds -c ../config/sensors_cfg.json`
- `start`
- `bt`
- `frame 1`
- `print *cmd`
- `disassemble`
- `info reg`

### Ejercicio 3
- Descomentar check (`cmd->execute == NULL`) en `command_runner.c:174`
- Reconstruir sensor_commands
- `gdb --args ./tools/sensor_cmds/sensor_cmds -c ../config/sensors_cfg.json`
  - `start`
  - `br sensor_command_experiment`
  - next hasta inicializar commands (linea 77)
  - `watch commands[2]`
  - `continue`


  - `print *cmd`
  - `print i`
  - `print *commands[i]`